#### Domande
- il numero di immagini è corretto?
- Dobbiamo fare object detection? E quindi classificare il dataset con i bounding box o non è necessario?
- Alla fine di tutto come dovremmo utilizzare il modello? Su una web-app ?

- Cambiare il dataset come visto in lezione 5
- Partiamo dal presupposto che abbiamo fatto training da modelli inizializzati randomicamente prima
- Uno fa alexNet e uno squeezenet (pretrained)
- Vediamo la differenza tra alexNet pretrained e no (vecchio codice) anche se cmq i layer del modello sono diversi.
- DA PULIRE CODICE .PY che estrae i frame
- Dobbiamo sistemare la nostra dev std o quella del modello?
- leggi domanda su paragrafo squeezenet

# Preparazione del dataset

In [84]:
import numpy as np
from PIL import Image
import random
from sklearn.model_selection import train_test_split
from torch.utils import data

In [1]:
from google.colab import drive
import zipfile

drive.mount('/content/gdrive')

Mounted at /content/gdrive


Tutte le labels sono presenti in un file txt chiamato all_labels

In [20]:
from glob import glob
from os.path import basename

all_labels_txt = '/content/gdrive/MyDrive/dataset_progetto/labels/all_labels.txt'

path_images = '/content/gdrive/MyDrive/dataset_progetto/img/'

# elenco delle classi

dataset_from_txt = np.loadtxt(all_labels_txt, dtype=str, delimiter=',')

In [54]:
all_images = []
all_labels = []

for i in range(len(dataset_from_txt)):
  all_images.append(path_images + dataset_from_txt[i][0])
  all_labels.append(int(dataset_from_txt[i][1]))

In [56]:
import pandas as pd

dataset = pd.DataFrame({'path': all_images, 'label': all_labels})
dataset.head()

,path,label
0,/content/gdrive/MyDrive/dataset_progetto/img/t...,0
1,/content/gdrive/MyDrive/dataset_progetto/img/t...,0
2,/content/gdrive/MyDrive/dataset_progetto/img/t...,0
3,/content/gdrive/MyDrive/dataset_progetto/img/t...,0
4,/content/gdrive/MyDrive/dataset_progetto/img/t...,0


Implemento un dizionario che mappi le classi su id numerici:

In [80]:
dictionary_dataset = {
    0: "empty",
    1: "half",
    2: "full"
}

# verifico di aver caricato tutto correttamente
#Image.open(dictionary_dataset['path'][234])
#dictionary[dictionary_dataset['label'][234]]

Il dataframe adesso contiene tutte le informazioni per caricare i dati

In [71]:
def split_train_val_test(dataset, perc=[0.6, 0.1, 0.3]):
  assert sum(perc)==1
  train, testval = train_test_split(dataset, test_size = perc[1]+perc[2])
  val, test = train_test_split(testval, test_size = perc[2]/(perc[1]+perc[2]))
  return train, val, test

In [78]:
random.seed(12345)
np.random.seed(12345)
train, val, test = split_train_val_test(dataset)

print("training:", len(train), "\nvalidation", len(val), "\ntest", len(test) )

training: 540 
validation 90 
test 270


Salvo i tre dataFrame su csv

In [79]:
train.to_csv('/content/gdrive/MyDrive/dataset_progetto/train.csv', index=None)
val.to_csv('/content/gdrive/MyDrive/dataset_progetto/valid.csv', index=None)
test.to_csv('/content/gdrive/MyDrive/dataset_progetto/test.csv', index=None)

salvo una corrispondenza tra le classi in un altro file csv

In [102]:
ids, classes = zip(*dictionary_dataset.items())
ids = pd.DataFrame({'id':ids, 'class':classes}).set_index('id')
ids.to_csv('/content/gdrive/MyDrive/dataset_progetto/classes.csv')

In [103]:
ids

,class
id,
0,empty
1,half
2,full


Creo una classe per caricare il csv

In [85]:
class TrashbeanDataset(data.Dataset):
  def __init__(self, csv, transform=None):
    self.data = pd.read_csv(csv)
    self.transform = transform

  def __len__(self):
    return len(self.data)
  
  def __getitem__(self, i):
    im_path, im_label = self.data.iloc[i]['path'], self.data.iloc[i].label
    
    im = Image.open(im_path)

    if self.transform is not None:
      im = self.transform(im)

    return im, im_label

Carico le classi da file e trasformo il dataframe in un dizionario

In [104]:
classes = pd.read_csv('/content/gdrive/MyDrive/dataset_progetto/classes.csv').to_dict()['class']

Carico il dataset dal csv

In [88]:
dataset_train = TrashbeanDataset('/content/gdrive/MyDrive/dataset_progetto/train.csv')
dataset_valid = TrashbeanDataset('/content/gdrive/MyDrive/dataset_progetto/valid.csv')
dataset_test = TrashbeanDataset('/content/gdrive/MyDrive/dataset_progetto/test.csv')

Testo che sia visualizzato correttamente

In [ ]:
#im, lab = dataset_train[0]
#print('Class id:',lab, 'Class name:',classes[lab])
#im

# Codice per allenare il modello

In [106]:
from torch.optim import SGD
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score
from os.path import join

class AverageValueMeter():
  def __init__(self):
    self.reset()

  def reset(self):
    self.sum = 0
    self.num = 0
  
  def add(self, value, num):
    self.sum += value*num
    self.num += num

  def value(self):
    try:
      return self.sum/self.num
    except:
      return None

In [107]:
def trainval_classifier(model, train_loader, validation_loader, exp_name='experiment', lr=0.01, epochs=10, momentum=0.99, logdir='logs'):
    criterion = nn.CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr, momentum=momentum)

    # meters
    loss_meter = AverageValueMeter()
    acc_meter = AverageValueMeter()
    # writer
    writer = SummaryWriter(join(logdir, exp_name))
    # device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    ## definiamo un dizionario contenente i loader di training e test
    loader = {
        'train': train_loader,
        'validation': validation_loader
    }
    global_step = 0
    for e in range(epochs):
        # iteriamo tra due modalità: train e test
        for mode in ['train', 'validation']:
            loss_meter.reset(); acc_meter.reset()
            model.train() if mode == 'train' else model.eval()
            with torch.set_grad_enabled(mode=='train'): # abilitiamo i gradienti o solo in training
                for i, batch in enumerate(loader[mode]):
                    x=batch[0].to(device) # portiamoli su device corretto
                    y=batch[1].to(device)
                    output = model(x)

                    # aggiorniamo il global_step
                    # conterrà il numero di campioni visti durante il training
                    n = x.shape[0]  # n di elementi nel batch
                    global_step += n
                    l = criterion(output, y)

                    if mode=='train':
                        l.backward()
                        optimizer.step()
                        optimizer.zero_grad()

                    acc = accuracy_score(y.to('cpu'), output.to('cpu').max(1)[1])
                    loss_meter.add(l.item(), n)
                    acc_meter.add(acc,n)

                    # loggiamo i risultati iterazione per iterazione solo durante il training
                    if mode == 'train':
                        writer.add_scalar('loss/train', loss_meter.value(), global_step=global_step)
                        writer.add_scalar('accuracy/train', acc_meter.value(), global_step=global_step)

                        # una volta finita l'epoca sia nel caso di training che di test loggiamo le stime finali

                writer.add_scalar('loss/' + mode, loss_meter.value(), global_step=global_step)
                writer.add_scalar('accuracy/' + mode, acc_meter.value(), global_step=global_step)

                        ## conserviamo i pesi del modello alla fine di un ciclo di training e test

        torch.save(model.state_dict(), '%s-%d.pth'%(exp_name, e+1))
    return model

In [109]:
def test_classifier(model, loader):
  device = "cuda" if torch.cuda.is_available() else "cpu"
  model.to(device)
  predictions, labels = [], []
  for batch in loader:
    x = batch[0].to(device)
    y = batch[1].to(device)
    output = model(x)
    preds = output.to('cpu').max(1)[1].numpy()
    labs = y.to('cpu').numpy()
    predictions.extend(list(preds))
    labels.extend(list(labs))
  return np.array(predictions), np.array(labels)

# Squeezenet

Adattiamo il modello

In [111]:
from torch import nn
from torchvision.models import squeezenet1_0
model = squeezenet1_0(pretrained=True)
num_class = 3
model.classifier[1] = nn.Conv2d(512, num_class, kernel_size=(1, 1), stride=(1, 1))
model.num_classes = num_class

**il primo test sarà fatto su immagini 256x256 (ma le nostre immagini sono 1920x1080). In questo caso conviene fare il resize prima di allenare il modello o converebbe adattare una cnn???** **Inoltre le medie che stiamo applicando sono quelle del codice visto a lezione. In questo caso (noi già abbiamo calcolato le medie nel vecchio colab (RCORDA DI IMPORARE QUELLA PARTE DI CODICE), converebbe utilizzarle?)**

In [ ]:
from torchvision import transforms
train_transform = transforms.Compose([
  transforms.Resize(256),
  transforms.RandomCrop(224),
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
test_transform = transforms.Compose([
  transforms.Resize(256),
  transforms.CenterCrop(224),
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
from torch.utils.data import DataLoader

dataset_train = TrashbeanDataset('/content/gdrive/MyDrive/dataset_progetto/train.csv', transform=train_transform)
dataset_valid = TrashbeanDataset('/content/gdrive/MyDrive/dataset_progetto/valid.csv', transform=train_transform)
dataset_test = TrashbeanDataset('/content/gdrive/MyDrive/dataset_progetto/test.csv', transform=train_transform)

dataset_train_loader = DataLoader(dataset_train, batch_size=32, num_workers=2, shuffle=True)
dataset_valid_loader = DataLoader(dataset_valid, batch_size=32, num_workers=2)
dataset_test_loader = DataLoader(dataset_test, batch_size=32, num_workers=2)

Lanciamo il training

In [ ]:
squeezenet_v1 = get_model()

squeezenet_v1_finetuned = trainval_classifier(squeezenet_v1, dataset_train_loader, dataset_valid_loader, \
                                                      exp_name='squeezenet_v1_finetuning', lr=0.001, epochs=50)

Calcoliamo l'accuracy

In [ ]:
#predizioni di test del modello all'ultima epoch
squeezenet_v1_finetuned_predictions_test, dataset_labels_test = \ 
  test_classifier(squeezenet_v1_finetuned, dataset_test_loader)

print("Accuracy di Squeezenet_v1: %0.2f%%" % \ 
       (accuracy_score(dataset_labels_test, squeezenet_v1_finetuned_predictions_test)*100,))

# Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs